# EDA

### Step 1: Problem statement and data collection

In [13]:
import pandas as pd

total_data = pd.read_csv('/workspaces/JLL_Linear_Regression_Regilarized/data/raw/health_insurance_data.csv', sep=',')
total_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


### Step 2: Exploration and data cleaning

In [14]:
total_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 108 columns):
 #    Column                                                                         Non-Null Count  Dtype  
---   ------                                                                         --------------  -----  
 0    fips                                                                           3140 non-null   int64  
 1    TOT_POP                                                                        3140 non-null   int64  
 2    0-9                                                                            3140 non-null   int64  
 3    0-9 y/o % of total pop                                                         3140 non-null   float64
 4    19-Oct                                                                         3140 non-null   int64  
 5    10-19 y/o % of total pop                                                       3140 non-null   float64
 6    20-29         

*Once we have obtained this information, it is important that we are able to draw conclusions, such as the following:

-There are a total of 3140 rows (in this case, people) and 108 columns, among which we will find the target or class to predict later   
-We don´t have nulls values     
-The data has 106 numerical characteristics and 2 categorical characteristics.

In [15]:
total_data['TOT_POP'].value_counts()

TOT_POP
14523    3
41619    2
6968     2
2611     2
65936    2
        ..
10143    1
21899    1
13257    1
24644    1
6967     1
Name: count, Length: 3074, dtype: int64

***Eliminates Duplicates***

In [16]:
total_data.drop('fips', axis = 1).duplicated().sum()

0

*There aren´t duplicates*

*If there were duplicates:*

In [17]:
# total_data = total_data.drop_duplicates(subset = total_data.columns.difference([''])) --> Column with ID
# print(total_data.shape)
# total_data.head()

***Eliminate irrelevant information***

In [18]:
total_data.drop(['COUNTY_NAME', 'STATE_NAME'], axis=1, inplace=True)

***We delete the categorical variables since we want to normalize the dataset to make a quick selection of interesting features***

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_features = scaler.fit_transform(total_data)

# Crear un nuevo DataFrame con las variables numéricas escaladas
total_data_scal = pd.DataFrame(norm_features, index = total_data.index, columns = total_data.columns)
total_data_scal["diabetes_number"] = total_data["diabetes_number"]
total_data_scal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 106 entries, fips to Urban_rural_code
dtypes: float64(105), int64(1)
memory usage: 2.5 MB


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

X = total_data_scal.drop(columns=["diabetes_number"])
y = total_data_scal["diabetes_number"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
train_indices = list(X_train.index)
test_indices = list(X_test.index)

k = int(len(X_train.columns) * 0.3)
selection_model = SelectKBest(score_func = f_regression, k = k)
selection_model.fit(X_train, y_train)
ix = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns = X_train.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns = X_test.columns.values[ix])

X_train_sel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 31 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   TOT_POP                                               2512 non-null   float64
 1   0-9                                                   2512 non-null   float64
 2   19-Oct                                                2512 non-null   float64
 3   20-29                                                 2512 non-null   float64
 4   30-39                                                 2512 non-null   float64
 5   40-49                                                 2512 non-null   float64
 6   50-59                                                 2512 non-null   float64
 7   60-69                                                 2512 non-null   float64
 8   70-79                                                 2512

***We make a quick selection of data by taking our target and applying the SelectionKBest model to make an EDA from it***

In [21]:
X_train_sel["diabetes_number"] = list(y_train)
X_test_sel["diabetes_number"] = list(y_test)

X_train_sel.to_csv("../data/processed/clean_train.csv", index = False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index = False)
total_data = pd.concat([X_train_sel, X_test_sel])
total_data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,Heart disease_number,COPD_number,CKD_number,diabetes_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.232975,-0.225814,-0.223516,-0.219329,2823
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.155304,-0.115392,-0.110080,-0.130962,5416
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.201976,-0.216207,-0.193106,-0.206391,3698
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,0.006163,-0.007276,-0.007077,-0.045054,7913
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.194913,0.097767,0.161314,0.088485,0.165555,0.182740,0.197812,0.265603,0.132454,12987
